In [1]:
import math
import time
import pandas as pd

# Define the function that implements the hybrid algorithm
def blendBF(f, a, b, eps=10**(-10)):
    # Initialize the variables
    n = 0
    a1 = a
    a2 = a
    b1 = b
    b2 = b

    while True:
        # Increment the iteration counter
        n += 1
        # Evaluate the function at the endpoints
        fa = f(a)
        fb = f(b)

        # Compute the midpoint and the false position point
        xB = (a + b) / 2
        fxB = f(xB)

        xF = a - (fa * (b - a)) / (fb - fa)
        fxF = f(xF)

        # Choose the one with the smaller absolute value as the root approximation
        if abs(fxB) < abs(fxF):
            x = xB
            fx = fxB
        else:
            x = xF
            fx = fxF

        # Check if the absolute value of fx is less than or equal to the tolerance
        if abs(fx) <= eps:
            # Return the output
            return x, fx, n, a, b

        # Update the interval by applying the bisection and false position methods
        if fa * fxB < 0:
            b1 = xB
        else:
            a1 = xB

        if fa * fxF < 0:
            b2 = xF
        else:
            a2 = xF

        # Set a to the maximum of a1 and a2 and b to the minimum of b1 and b2
        a = max(a1, a2)
        b = min(b1, b2)

# Read the file and store the equations and the intervals in two lists
functions = []
intervals = []
with open('functions.txt', 'r') as file:
    for line in file:
        equation, a, b = line.split(',')  # Split the line by commas
        # Convert the equation to a lambda function and append to the list
        functions.append(eval(f'lambda x: {equation}'))
        # Convert the bounds to floats and append to the list
        intervals.append([float(a), float(b)])

# Define the number of runs for each function
num_runs = 500

# Create a list of tuples containing the function, the interval, and the average time for the hybrid algorithm
results = []
for i in range(10):
    f = functions[i]  # Get the function from the list
    # Get the interval from the list and use different variables
    a1, b1 = intervals[i]
    total_time = 0
    for j in range(num_runs):
        start_time = time.time()
        # Use the new variables as arguments
        result, fx, iter, x0, x1 = blendBF(f, a1, b1)
        end_time = time.time()
        run_time = end_time - start_time
        total_time += run_time

    average_time = total_time / num_runs
    # Use the new variables in the list
    results.append((f, [a1, b1], average_time))

# Create a pandas dataframe from the results list
df = pd.DataFrame(results, columns=['Function', 'Interval', 'Avg CPU Time'])

# Add columns for the root, the function value, the iterations, the lower bound, and the upper bound, using the blendBF function
df['Root'] = df.apply(lambda row: blendBF(row['Function'], row['Interval'][0], row['Interval'][1])[0], axis=1)
df['f(x)'] = df.apply(lambda row: blendBF(row['Function'],row['Interval'][0], row['Interval'][1])[1], axis=1)
df['Iter'] = df.apply(lambda row: blendBF(row['Function'], row['Interval'][0], row['Interval'][1])[2], axis=1)
# Add the lower bound column
df['a'] = df.apply(lambda row: blendBF(row['Function'], row['Interval'][0], row['Interval'][1])[3], axis=1)
# Add the upper bound column
df['b'] = df.apply(lambda row: blendBF(row['Function'], row['Interval'][0], row['Interval'][1])[4], axis=1)

# Drop the function and interval columns
df = df.drop(['Function', 'Interval'], axis=1)

# Add the problem column with labels
df['Problem'] = [f'$P{i+1}$' for i in range(10)]

# Reorder the columns
# Include the lower and upper bound columns
df = df[['Problem', 'Iter', 'Avg CPU Time', 'Root', 'f(x)', 'a', 'b']]

# Rename the columns
df = df.rename(columns={'Root': 'Approximate Root', 'f(x)': 'Function Value', 'a': 'Lower Bound', 'b': 'Upper Bound'})

# Use the scientific notation format for the Avg CPU Time column
pd.options.display.float_format = ' {:.12e}'.format

# Save the dataframe to an excel file with a different name, such as hybrid-table.xlsx
df.to_excel('hybrid-table.xlsx', index=False)

# Display the dataframe in Jupyter Notebook
df

,Problem,Iter,Avg CPU Time,Approximate Root,Function Value,Lower Bound,Upper Bound
0,$P1$,9,5.605697631836e-05,1.365230013414e+00,-5.250910817267e-12,1.365230013271e+00,1.369770382784e+00
1,$P2$,1,4.209518432617e-06,2.000000000000e+00,0.000000000000e+00,0.000000000000e+00,4.000000000000e+00
2,$P3$,8,2.782058715820e-05,6.931471805540e-01,-1.181099662517e-11,6.931471790930e-01,7.012092856566e-01
3,$P4$,5,9.454727172852e-06,3.141592653605e+00,-1.509491386733e-11,3.141590357956e+00,3.147874957381e+00
4,$P5$,1,5.058288574219e-06,1.000000000000e+00,0.000000000000e+00,1.000000000000e+00,2.500000000000e+00
5,$P6$,1,4.219532012939e-06,-2.000000000000e+00,0.000000000000e+00,-2.500000000000e+00,-1.500000000000e+00
6,$P7$,6,1.050186157227e-05,7.390851332053e-01,1.650546366250e-11,7.390851296377e-01,7.516528968220e-01
7,$P8$,1,1.999855041504e-06,3.000000000000e+00,0.000000000000e+00,2.000000000000e+00,4.000000000000e+00
8,$P9$,1,0.000000000000e+00,0.000000000000e+00,0.000000000000e+00,-1.000000000000e+00,1.000000000000e+00
9,$P10$,6,4.638624191284e-05,3.111748656315e+00,-4.813216492039e-11,3.098905999443e+00,3.111748659071e+00
